## Huffman Encoding
1. Initalise all unique ASCII characters as Huffman nodes
2. Hold a priority queue/min heap holding all elements with their key being their frequency
3. While the min heap has a size of greater than 1
   1. Pop the top two elements out
   2. Create a new huffman node from these elements 
   3. Add this node to the min heap
4. Do a DFS of the remaining Huffman node, returning a dictionary of each leaf node symbol: recursive stack 

In [30]:
import typing
import heapq
from collections import defaultdict
from bitarray import bitarray
## Finish type hings
class HuffmanNode():
    def __init__(self, value: str = None, frequency: int = 0, right  = None, left = None):
        self.value = value
        self.frequency = frequency
        self.right = right
        self.left = left

    def __lt__(self, other):
        """Compare nodes based on frequency for heapq."""
        return self.frequency < other.frequency

    def __str__(self) -> str:
        #return f" Value: {self.value} Frequency: {self.frequency}"
        return f" Value: {self.value} \nFrequency: {self.frequency}\nRight: {self.right}\nLeft: {self.left}"



def retrieve_codes(current, codes, path : str):
    '''Conducts DFS on a HuffmanNode returning the stack (binary encoding) and leaf value
    As no cycles exist in a tree, no need for a visited data structure'''
    ## base case have hit a leaf
    if not current.right: ## can just check for right child, as all nodes will either have no children or both left and right
        codes[current.value] = path
        return codes
    
    ## traverse the children 
    codes = retrieve_codes(current = current.left, codes = codes, path = path + "0")
    codes = retrieve_codes(current = current.right, codes = codes, path = path + "1")
    return codes

    
    
def huffman_encode(str:str):
    if len(str) == 0:
        return
    # Create a hashmap in order to find all unique ASCII characters in str, ensure each 
    # new character has a new Huffman node
    chars = defaultdict(lambda: HuffmanNode(value = None)) ## Maps characters to huffman nodes

    for c in str:
        if chars[c].value is None:
            chars[c].value = c
        chars[c].frequency += 1 ## Increment the frequency of the given entry 

    min_freq = list(chars.values())
    heapq.heapify(min_freq)

    ## Coalesce elements until only one remains - the root node
    while len(min_freq) > 1: ## potentially inefficient o(n)
        ## pop off two least frequent elements
        least = heapq.heappop(min_freq)
        second = heapq.heappop(min_freq)

        name = least.value + second.value
        freq = least.frequency + second.frequency
        combined = HuffmanNode(value = name, frequency = freq, left = least, right = second)

        heapq.heappush(min_freq, combined)

    root = min_freq.pop()

    ## potentially can make a predecessor array to backtrack
    codes = defaultdict(lambda: "")
    ## perfrom dfs from the root node to return all character encodings
    return dict(retrieve_codes(current = root, codes = codes, path = ""))

print(huffman_encode("A_DEAD_DAD_CEDED_A_BAD_BABE_A_BEADED_ABACA_BED"))

{'D': '00', '_': '01', 'A': '10', 'E': '110', 'C': '1110', 'B': '1111'}


In [26]:
chol = bitarray()

print(chol + bitarray(0))

bitarray()


## Elias Gamma Encoding
##### Encoding
1. Find the largest $N$ with $2^n = x$ (greatest power of $2$)
2. Encoding $N$ using unary coding (with zeros) 
3. Append the integer $x - 2^N$ using $N$ digits in binary

> Example : Encoding $10$
> 1. $2^3$ = $8$ 
> 2. $N = 3$. $3$ in unary encoding = `000`, add one to obtain `0001`
> 3. $10-8 = 2$, therefore encoding $2$ in binary: `010`
> Therefore $10$ = `0001010`

##### Decoding
1. Decode unary code at the beginning (count first zeroes) this is our $N$
2. Read the next $N+1$ bits as a binary number, that corresponds to $x-2^N$

 Example : Decoding `num =` `0001010`
> 1. Unary code is all zeros before first 1 : `000`, equals 3. $\therefore N = 3$
> 2. `num[N+1:]` is `010`. This represents $2$
> 3. Therefore `num` = $2^N + 2 = 2^3 +2 = 10$

## Elias Omega Encoding
Balances the sizes of very different integers. Use case: when need to store input data in a very large integer range. 
Will represent integers closer to 0 and very far from 0 with a more balanced/equal allocation of space. 
##### Encoding:
$L_n...L_3, L_2, L_1, N$ 
Calculated recursively. Continues until $|L_i| = 1$. Initial case is $L1 = |N|$ (number of bits in $N$)
1. Calculate $L_i$ = $|L_{i-1}|$ with the leading bit flipped from $1$ to $0$
2. Prepend $L_i$ to the encoding / recursively add innermost stack call return to gradually outer stack call returns 


In [2]:
import typing 
from bitarray import bitarray

def elias(n: int):
    '''
    elias returns elias encoding for a given decimal integer
    '''
    ## recursive function
    def add_code(l: bitarray) -> bitarray:
        ## If the length component is equal to 1, return 
        if len(l) == 1:
            return l
        
        ## Recursive case, calculate the next smallest length component to be prepended
        l_length : int = (len(l)-1) 

        ## Convert it to binary and flip the leading bit
        next_l : bitarray  = bitarray(bin(l_length)[2:])
        next_l[0] = 0
        return add_code(next_l) + l
    
    ## Convert n to binary (removing '0f' prefix)
    n_binary : bitarray = bitarray(bin(n)[2:])
    return add_code(n_binary).to01() ## Converts it to string form

print(elias(561))


00100011000110001



##### Decoding

Starting `component` = `encoded[0]`.
1. Read component. If it `component[0]` is $0$, this is a length component. 
   1. Switch the first bit such that `component[0]` = 1
   2. Read component. Value of component = `|next_component|` + 1 
   3. Shift to `next_component` which exists at the position after `component` `i` to `i + |next_component|`
   4. Continue 
2. If component[0] = 1 this indicates that this component is $N$ so can read $N$.

Elias Omega Encoding guarantees that each number represented has a unique prefix - as it recursively encodes each order of magnitude. 
Idea of mapping from the positive integer set to the countable integer set

In [6]:
import typing 
from bitarray import bitarray

def decode_elias(code: str):
    '''
    decode_elias recursively and efficiently decodes a binary string encoded in elias omega
    '''
    ## convert code to bitarray
    code = bitarray(code)

    ## recursive function
    def return_start_n(i: int, length = int) -> int:
        ## If the first bit is 1, i points to the start of N
        if code[i] == 1:
            return i
        
        ## Else if the first bit is a 0, this is a length component to traverse over
        code[i] = 1 # Flip the first bit of this length component to 1
        next_length = int((code[i : i + length]).to01(),2) # Calculate the length of the following length component
        next_length += 1 # Add one to offset the -1 done in the encoding

        ## Calculate the next position of i
        next_i = i + length # After having processed this length component, shift i rightwards
        return return_start_n(i = next_i, length = next_length)
    
    ## first component is always a length component = 0 with size = 1
    start = return_start_n(i = 0, length = 1) 
    return int(code[start:].to01(),2) #code[start:] 


for i in range(1,10000):
    # print(i)
    assert i == decode_elias(elias(i))
print("All Correct!")

All Correct!


# LZ77 Compression
Window comprises of two consecutive parts
1. Search window ('dictionary')
2. Lookahead buffer ('buffer)

To encode any substring in the lookahead buffer, find the largest matched substring in the dictionary
1. The offset (distance of the match from the start position of the match from the current substring being encoded)
2. Lenght of the match
3. The next character `char` in the lookahead buffer `(k+1)`

Using this search, the substring at the current position is encoded as a triple `<offset, length, char>` 

Note that this triple is not stored as `<integer, integer, character>`

It is stored as variable length bits - `<elias encoding, elias encoding, huffman encoding>`

Everything time shift `j` up (`j` is where we are currently looking within the lookahead buffer) we ask the question 
> what is the longest substring starting at that position that matches in the past?

- Let `b` = size of lookahead buffer
- Let `s` = size of search window 
The size of the lookahead buffer, call it `b` is a parameter, setting it to greater values will trade speed for space (takes far longer to evaluate matches, as they are truncated at a longer length, but resulting compression will take up much less space). Thus the lookahead buffer bounds the length of the match found between `str[j....j+b]`.
This match can occur between `str[j-s...j-s+b]`, so only matches of up to size `b` are compressed

```python
triple = [offset = 0, len = 0, next_char = str[0]]
1. While j < n
    1. Find the longest matched substring starting after search window and matching j as much as possible up until j+b
        1. If one exists, update triple
   1. j = j + offset ## move past the largest match made at j 

```

What remains after the LZ77 Encoding is an array, or some collection of triples


In order to further optimise, LZSS, uses a tuple `<bit-flag = bit-0, current_char>`  

rather than a triple `<bit-flag = bit-0, offset, length>` 

for all characters which have no previous match

# Encoder and Decoder LZ77
Given a string `str = ababab$`
And a search window of a given size `sw`
The resultant triples encoding `str` would be
1. `<0,0,'a'>`
2. `<0,0,'b'>`
3. `<2,4,'$'>` 

> Note that the triples always encode one letter greater than what we read
Each triple will be encoded as
`<Elias-Start, Elias-Length, Huffman-Character>`

And the huffman codes in the preamble will be encoded as
`<ASCII-Code, Elias-Length, Huffman-Code-Character>`
Where each of these triples describes the huffman code for a given character (in lexicographical order, doesn't matter)
Where ASCII is always one byte, elias-length is self-encoded/contained, huffman-character is variable length
Note here that the elias-length is the length of the huffman code for this given character
This will be read by the decoder and each triple inserted into a hashmap for quick look up/decoding

If the decoder will construct the huffman encodings itself, rather than encoding it in the encoder step, the preamble constituents instead can be 
`<ASCII-Code, Elias-Frequency>`
Where Elias-Frequency describes the frequency of the given ASCII character

The resultant output file for this encoding will contain:
```python
1. Elias encoding for the length of the preamble (so decoder knows where to read up to)
2. Preamble triples <ASCII, EliasLength, HuffmanCode>
3. LZ77 Triples <EliasStart, EliasLength, HuffmanCode>
```

Encoder Function: 
Parameters: 
- `sw` which is the search window size
- `bs` which is the buffer size

Pointers in Iteration:
1. `i`: one in the str context holding the current position of str that we are encoding
2. `b`: and one starting at the buffer window (this is calculated as `i - bs`)
3. (tentative, implicit) `s`: `i + sw`

Idea is that we want the maximal match/the greatest matched prefix from `str[i...s]`
that matches something occuring after `str[b]` (can this occur all the way up to `[i...s]`?)

potential optimisations i thought about but failed
1. calculating the huffman codes from the the triples rather than from the string
not space efficient as have to store all the triples associated with the entire document (e.g. all 100000 pages) to calculate the huffman codes for just the characters afterwards. 
2. only including the huffman encoding of characters occuring in the triples in the preamble, rather than all huffman encodings
every character in the string will occur as at least ONCE in the sequence of triples (as for the first occurence of that character, nothing to match to occuring before it) 

```python
## calculate all huffman codes from the original string
huffman_codes : hashmap = huffman(str)
encoding = bitarray()
## Constructing LZ77 Triples
while i < len(str):
    buffer = str[b...i]
    search = str[i...s]
    use modified z algorithm on the compound string : search + "$" + buffer + search
        find the greatest z value occuring within the buffer portion of this compound string
        if there a multiple matching values, tie breaker is the leftmost one, as minimises elias encoding for offset.add(str[i])
    new triple = <offset (position of chosen z value), maximum z value, huffman_codes[str[i]]>
    encoding.append(binary(new_triple)) ## add binary value of new triple to encoding
    i += length

## Constructing the preamble
preamble_length = 0
For each character, code in huffman_codes:
    elias_length = elias(len(code))
    ascii_code = ascii(character)
    triple = <ascii_code, elias_length, code> ## encode this as a bitarray

    preamble_length += length(triple)
    encoding.prepend(triple)

## Prepend the length of the preamble
preamble_length  = binary(preamble_length) 
preamble_length = preamble_length + length(preamble_length) ## offset by the length of preamble length
encoding.prepend(preamble_length)

## Construct the encoding 
return encoding ## or write encoding to a text file
```
Research into how bitarrays are stored - if they are linked list, prepending and appending are efficient
Potential optimisation - instead of doing a naive scan for the first and second character, know the first triple is

Additional AND FEASIBLE optimisation:
``` python
last_seen = [0 for i in range (ascii range)]
1. Create a character indexed (last seen) array, which holds the last given occurence for a given character. When we encounter a new character, check if it has occured within the search window:
if last_occurrence[character] is within i - buffer size e.g     [i-b]    occurs here   [i]
then need to do z box
however, if last occurrence is not within current sliding window, then skip z algorithm and instead
immediately insert the triple <0,0,character>

This prevents unnecessary z algorithm being done
```

May not actually do anything given small alphabet size (e.g. with DNA), large search window (based on hardware)
